In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, confusion_matrix,log_loss, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
import nltk
import gensim
import os
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv('PS3_training_data.txt',  delimiter='	', names = ["index","sentence","polarity","issue","genre", "uname"])
data.head()

,index,sentence,polarity,issue,genre,uname
0,0,This is definitely a must have if your state d...,POSITIVE,NONE,GENRE_B,NaN
1,1,It's a great place and I highly recommend it.,POSITIVE,NONE,GENRE_B,NaN
2,2,"I will see the doctors, take some blood tests ...",NEUTRAL,GOING_TO_PLACES,GENRE_A,NaN
3,3,I can tell you about having my phone and elect...,NEGATIVE,MONEY_ISSUE,GENRE_A,NaN
4,4,Their steaks are 100% recommended!,POSITIVE,NONE,GENRE_B,NaN


In [4]:
data = data[["sentence","polarity","issue","genre"]]
# hist = data.hist(bin = 3)
data.head()


,sentence,polarity,issue,genre
0,This is definitely a must have if your state d...,POSITIVE,NONE,GENRE_B
1,It's a great place and I highly recommend it.,POSITIVE,NONE,GENRE_B
2,"I will see the doctors, take some blood tests ...",NEUTRAL,GOING_TO_PLACES,GENRE_A
3,I can tell you about having my phone and elect...,NEGATIVE,MONEY_ISSUE,GENRE_A
4,Their steaks are 100% recommended!,POSITIVE,NONE,GENRE_B


In [5]:
POS = len(data[data.polarity == "POSITIVE"])
NEG = len(data[data.polarity == "NEGATIVE"])
NEU = len(data[data.polarity == "NEUTRAL"])
total = len(data)
print(" POS : % , NEG : %, NEU : %",(POS/total,NEG/total,NEU/total))

 POS : % , NEG : %, NEU : % (0.41953125, 0.50078125, 0.0796875)


In [41]:
# class MySentences(object):
#     """MySentences is a generator to produce a list of tokenized sentences 
    
#     Takes a list of numpy arrays containing documents.
    
#     Args:
#         arrays: List of arrays, where each element in the array contains a document.
#     """
#     def __init__(self, *arrays):
#         self.arrays = arrays
 
#     def __iter__(self):
#         for array in self.arrays:
#             for document in array:
#                 for sent in nltk.sent_tokenize(document):
#                     yield nltk.word_tokenize(sent)

# def get_word2vec(sentences, location,retrain):
#     """Returns trained word2vec
    
#     Args:
#         sentences: iterator for sentences
        
#         location (str): Path to save/load word2vec
#     """
#     if os.path.exists(location) and not retrain :
#         print('Found {}'.format(location))
#         model = gensim.models.Word2Vec.load(location)
#         return model
    
#     print('{} not found. training model'.format(location))
#     model = gensim.models.Word2Vec(sentences, size=100, window=5, min_count=5, workers=4)
#     print('Model done training. Saving to disk')
#     model.save(location)
#     return model

# corpus_train = data["sentence"]
# w2vec = get_word2vec(
#     MySentences(
#        corpus_train
#     ),
#     'w2vmodel.model',
#     False
# )



In [42]:
# class MyTokenizer:
#     def __init__(self):
#         pass
    
#     def fit(self, X, y=None):
#         return self
    
#     def transform(self, X):
#         transformed_X = []
#         for document in X:
#             tokenized_doc = []
#             for sent in nltk.sent_tokenize(document):
#                 tokenized_doc += nltk.word_tokenize(sent)
#             transformed_X.append(np.array(tokenized_doc))
#         return np.array(transformed_X)
    
#     def fit_transform(self, X, y=None):
#         return self.transform(X)

# class MeanEmbeddingVectorizer(object):
#     def __init__(self, word2vec):
#         self.word2vec = word2vec
#         # if a text is empty we should return a vector of zeros
#         # with the same dimensionality as all the other vectors
#         self.dim = len(word2vec.wv.syn0[0])

#     def fit(self, X, y=None):
#         return self

#     def transform(self, X):
#         X = MyTokenizer().fit_transform(X)
        
#         return np.array([
#             np.mean([self.word2vec.wv[w] for w in words if w in self.word2vec.wv]
#                     or [np.zeros(self.dim)], axis=0)
#             for words in X
#         ])
    
#     def fit_transform(self, X, y=None):
#         return self.transform(X)
    
# mean_embedding_vectorizer = MeanEmbeddingVectorizer(w2vec)

In [43]:
# cv = CountVectorizer().fit(data["sentence"])
df_train,df_test = train_test_split(data,test_size=0.2, random_state=35)



In [44]:
# x_train = mean_embedding_vectorizer.fit_transform(df_train['sentence'])
# y_train = df_train['polarity']
# x_test = mean_embedding_vectorizer.fit_transform(df_test['sentence'])
# y_test = df_test['polarity']
# print((x_train.shape,y_train.shape))
# ros = RandomOverSampler(random_state=42)
# x_train,y_train =  ros.fit_resample(x_train,y_train)
# print((x_train.shape,y_train.shape))

In [45]:
x_train = cv.transform(df_train['sentence'])
y_train = df_train['polarity']
x_test = cv.transform(df_test['sentence'])
y_test = df_test['polarity']
print((x_train.shape,y_train.shape))
ros = RandomOverSampler(random_state=42)
x_train,y_train =  ros.fit_resample(x_train,y_train)
print((x_train.shape,y_train.shape))

((2048, 6689), (2048,))
((3078, 6689), (3078,))


In [46]:
model = RandomForestClassifier(n_estimators=100, max_depth=2,random_state=0)
model.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [47]:

predict = model.predict(x_test)
print(predict.shape)
print(y_test.shape)

(512,)
(512,)


In [48]:
print(accuracy_score(y_test, predict))

0.5859375


In [49]:
print(classification_report(y_test, predict))

              precision    recall  f1-score   support

    NEGATIVE       0.69      0.56      0.62       256
     NEUTRAL       0.30      0.51      0.38        47
    POSITIVE       0.59      0.63      0.61       209

   micro avg       0.59      0.59      0.59       512
   macro avg       0.53      0.57      0.54       512
weighted avg       0.61      0.59      0.59       512



In [40]:
print(confusion_matrix(y_test, predict))

[[231   0  25]
 [ 46   0   1]
 [180   0  29]]
